In [ ]:
import sys
# setting path
sys.path.append('../')

In [ ]:
import matplotlib.pyplot as plt
import os
SAVE = True
SAVE_PATH = '/work/work_fran/SmoothAttention/results/neurips_figures/panda_heatmaps/'
FIGSIZE = (10,10)
SAVE_EXTENSION = 'png'

os.environ['PATH'] = os.environ['PATH'] + ':/home/fran/latex/bin/x86_64-linux/'
plt.rcParams.update({"text.usetex": True, 'font.size': 20})

In [ ]:
from datasets import PandaDataset

SIZE = 512
RESIZE_SIZE = 256
DATA_DIR = f'/data/data_fran/Panda/patches_{SIZE}'

data_path = f'/data/data_fran/Panda/patches_{SIZE}/raw/features_resnet18/'
processed_data_path = f'/data/data_fran/Panda/patches_{SIZE}/processed/features_resnet18/'
# csv_path = f'/data/data_fran/Panda/patches_{SIZE}/train_patches.csv'
csv_path = f'/data/data_fran/Panda/patches_{SIZE}/test_patches.csv'

dataset = PandaDataset(data_path=data_path, processed_data_path=processed_data_path, csv_path=csv_path, use_patch_distances=False)

In [ ]:
bag_len_list = []
for bag_name in dataset.bag_names:
    bag_len_list.append(len(dataset.data_dict[bag_name]['inst_names']))

In [ ]:
import numpy as np
idx_bag_names_sorted = np.argsort(bag_len_list)

In [ ]:
idx_pos_bags_sorted = []
for idx in idx_bag_names_sorted:
    if dataset.data_dict[dataset.bag_names[idx]]['bag_label'] == 1:
        idx_pos_bags_sorted.append(idx)
print(len(idx_pos_bags_sorted))

In [ ]:
BAG_IDX = idx_pos_bags_sorted[1000]
WSI_NAME = dataset.bag_names[BAG_IDX]
WSI_LABEL = dataset.data_dict[WSI_NAME]['bag_label']
PATCH_NAMES = dataset.data_dict[WSI_NAME]['inst_names']
PATCH_LABELS = dataset.data_dict[WSI_NAME]['inst_labels']
print(WSI_NAME)
print(len(dataset.data_dict[WSI_NAME]['inst_names']))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm


patches_list = []
row_list = []
column_list = []
pbar = tqdm(total=len(PATCH_NAMES))
for patch_name in PATCH_NAMES:
    pbar.update(1)
    img = cv2.imread(f'{DATA_DIR}/images/' + patch_name + '.jpg')
    # img = cv2.imread(f'{DATA_DIR}/images/' + patch_name + '.png')
    if RESIZE_SIZE != SIZE:
        img = cv2.resize(img, (RESIZE_SIZE, RESIZE_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    patches_list.append(img)
    row = int(patch_name.split('_')[1])
    column = int(patch_name.split('_')[2])
    # print('NC:', df[df['image_name'] == image_name]['NC'].values[0])
    # print('unlabeled:', df[df['image_name'] == image_name]['unlabeled'].values[0])
    # print('label:', label)
    row_list.append(row)
    column_list.append(column)
ROW_ARRAY = np.array(row_list)
COL_ARRAY = np.array(column_list)
min_row = np.min(ROW_ARRAY)
min_column = np.min(COL_ARRAY)
ROW_ARRAY = ROW_ARRAY - min_row + 1
COL_ARRAY = COL_ARRAY - min_column + 1
MAX_ROW = np.max(ROW_ARRAY)
MAX_COL = np.max(COL_ARRAY)
print(MAX_ROW, MAX_COL)

In [ ]:
MAX_X = (MAX_COL+2) * RESIZE_SIZE
MAX_Y = (MAX_ROW+2) * RESIZE_SIZE

canvas_wsi = np.zeros((MAX_Y, MAX_X, 3), dtype=np.uint8)+255

for i, patch in enumerate(patches_list):
    row = ROW_ARRAY[i]
    column = COL_ARRAY[i]
    x = column * RESIZE_SIZE
    y = row * RESIZE_SIZE
    canvas_wsi[y:y+RESIZE_SIZE, x:x+RESIZE_SIZE] = patch
    # canvas_attmap[y:y+RESIZE_SIZE, x:x+RESIZE_SIZE] = attmap[row-1, column-1]

# canvas_wsi = canvas_wsi.transpose(1, 0, 2)
# ROW_ARRAY, COL_ARRAY = COL_ARRAY, ROW_ARRAY

In [ ]:
from plot_utils import plot_wsi_and_heatmap
# fig, ax = plt.subplots(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10), nrows=2)
ax[0] = plot_wsi_and_heatmap(ax[0], canvas_wsi, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[1] = plot_wsi_and_heatmap(ax[1], canvas_wsi, PATCH_LABELS, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
plt.show()

In [ ]:
X, T, y, edge_index = dataset[BAG_IDX]
adj_mat = edge_index.to_dense().numpy()

In [ ]:

changes = False

old_patch_labels = PATCH_LABELS
while True:

    new_patch_labels = []
    changes = False
    for i in range(len(old_patch_labels)):
        sum_pos = 0
        sum_neg = 0
        for j in range(len(old_patch_labels)):
            if adj_mat[i, j] > 0:
                if old_patch_labels[j] == 1:
                    sum_pos += 1
                else:
                    sum_neg += 1
         
        if sum_pos > sum_neg:
            new_label = 1
        else:
            new_label = 0
        new_patch_labels.append(new_label)
        if new_label != old_patch_labels[i]:
            changes = True
    
    if not changes:
        break
    old_patch_labels = new_patch_labels

print("finished")


from plot_utils import plot_wsi_and_heatmap
# fig, ax = plt.subplots(figsize=(10,10))
fig, ax = plt.subplots(figsize=(10,10), ncols=2)
ax[0] = plot_wsi_and_heatmap(ax[0], canvas_wsi, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[1] = plot_wsi_and_heatmap(ax[1], canvas_wsi, new_patch_labels, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
plt.show()

In [ ]:
PATCH_LABELS = new_patch_labels

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# save ground truth labels
fig, ax = plt.subplots(figsize=(10,10))
ax = plot_wsi_and_heatmap(ax, canvas_wsi, PATCH_LABELS, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)

N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(0.17254901960784313, 0.8392156862745098, N)
vals[:, 1] = np.linspace(0.6274509803921569, 0.15294117647058825, N)
vals[:, 2] = np.linspace(0.17254901960784313, 0.1568627450980392, N)
cmap = ListedColormap(vals)

fig.tight_layout()

cbaxes = inset_axes(ax, width="30%", height="3%", loc='upper right') 
fig.colorbar(
    cm.ScalarMappable(norm=None, cmap=cmap), 
    cax=cbaxes, 
    orientation='horizontal',
    ticks=[0,1]
)
plt.show()

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [ ]:
run_path_dict = {
    'attpool' : 'francastro-team/SmoothAttention/8qgpcvdl',
    'smoothattpool' : 'francastro-team/SmoothAttention/5afwlfdq',
    'smoothtransformer' : 'francastro-team/SmoothAttention/xadho6za',
    'transmil' : 'francastro-team/SmoothAttention/aepg7pyr',
    'gtp' : 'francastro-team/SmoothAttention/xmfuxcar',
    'camil' : 'francastro-team/SmoothAttention/gok6nof2',
    'dsmil' : 'francastro-team/SmoothAttention/0qirflr5',
    'clam' : 'francastro-team/SmoothAttention/lfdkoqe2'
}

In [ ]:
import wandb
import argparse
from experiments.model_builder import build_MIL_model
import torch

X, T, y, edge_index = dataset[BAG_IDX]
adj_mat = edge_index.to_dense()
X = X.unsqueeze(0)
T = T.unsqueeze(0)
y = y.unsqueeze(0)
adj_mat = adj_mat.unsqueeze(0)
mask = torch.ones_like(y)

f_pred_dict = {}
attmap_dict = {}

for model_name in run_path_dict.keys():
    print(model_name)
    run_path = run_path_dict[model_name]
    api = wandb.Api()
    run = api.run(run_path)
    config = run.config
    config = argparse.Namespace(**config)

    model = build_MIL_model(config)
    model = model.to('cpu')

    weights = run.file('weights/best.pt').download(replace=True)

    weights_dict = torch.load(weights.name)

    model.load_state_dict(weights_dict)

    T_logits_pred, f_pred = model.predict(X, adj_mat, mask, return_y_pred=True)
    f_pred = f_pred.detach().cpu().squeeze(0).numpy()

    f_pred_dict[model_name] = f_pred

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), ncols=len(run_path_dict.keys()))

for i, model_name in enumerate(run_path_dict.keys()):
    f_pred = f_pred_dict[model_name]
    f_pred = normalize(f_pred)
    ax[i] = plot_wsi_and_heatmap(ax[i], canvas_wsi, f_pred, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
    ax[i].set_title(model_name)

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE)
ax = plot_wsi_and_heatmap(ax, canvas_wsi.transpose(1, 0, 2), new_patch_labels, size=RESIZE_SIZE, plot_patch_contour=True, row_array=COL_ARRAY, col_array=ROW_ARRAY, alpha=0.3)
if SAVE:
    plt.savefig(f'{SAVE_PATH}/panda_wsi_patched_labels_alpha.png', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE)
ax = plot_wsi_and_heatmap(ax, canvas_wsi, new_patch_labels, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY, alpha=0.8)
if SAVE:
    plt.savefig(f'{SAVE_PATH}/panda_wsi_patched_labels.png', bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


if SAVE:

    # save wsi image without patches
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax = plot_wsi_and_heatmap(ax, canvas_wsi, size=RESIZE_SIZE, plot_patch_contour=False, row_array=ROW_ARRAY, col_array=COL_ARRAY)
    plt.savefig(f'{SAVE_PATH}/panda_wsi.{SAVE_EXTENSION}', bbox_inches='tight')

    # save wsi image with patches
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax = plot_wsi_and_heatmap(ax, canvas_wsi, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
    plt.savefig(f'{SAVE_PATH}/panda_wsi_patched.{SAVE_EXTENSION}', bbox_inches='tight')

    # save ground truth labels
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax = plot_wsi_and_heatmap(ax, canvas_wsi, PATCH_LABELS, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
    
    N = 256
    vals = np.ones((N, 4))
    vals[:, 0] = np.linspace(0.17254901960784313, 0.8392156862745098, N)
    vals[:, 1] = np.linspace(0.6274509803921569, 0.15294117647058825, N)
    vals[:, 2] = np.linspace(0.17254901960784313, 0.1568627450980392, N)
    cmap = ListedColormap(vals)

    fig.tight_layout()

    cbaxes = inset_axes(ax, width="30%", height="3%", loc='upper right') 
    fig.colorbar(
        cm.ScalarMappable(norm=None, cmap=cmap), 
        cax=cbaxes, 
        orientation='horizontal',
        ticks=[0,1]
    )    
    plt.savefig(f'{SAVE_PATH}/panda_wsi_patched_labels_bar.{SAVE_EXTENSION}', bbox_inches='tight')

    # save model heatmaps
    for i, model_name in enumerate(run_path_dict.keys()):
        f_pred = f_pred_dict[model_name]
        f_pred = normalize(f_pred)
        fig, ax = plt.subplots(figsize=FIGSIZE)
        ax = plot_wsi_and_heatmap(ax, canvas_wsi, f_pred, size=RESIZE_SIZE, plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
        plt.savefig(f'{SAVE_PATH}/panda_wsi_patched_{model_name}.{SAVE_EXTENSION}', bbox_inches='tight')
    